Train SVM for each finger pair!\
Data source: \
sliding windowed powers for mu and beta band\
Subject 1

In [1]:
import os
import numpy as np
import h5py
import mne
from scipy import stats
import scipy.io

mne.set_log_level('error')

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [2]:
subject_id = 0
tag = 'reproduced'

In [3]:
# Load the dictionary from the HDF5 file
target_dir = 'features'
file_path = os.path.join(target_dir, tag + '_' + cfg['subjects'][subject_id] + '.h5')


data = {}
with h5py.File(file_path, 'r') as h5file:
    for key in h5file.keys():
        data[key] = np.array(h5file[key])

# Print the loaded data dictionary
for key, value in data.items():
    print(key, value.shape)

index (50, 153, 74, 4)
little (50, 153, 74, 4)
middle (50, 153, 74, 4)
ring (50, 153, 74, 4)
thumb (50, 153, 74, 4)


In [4]:
for key, value in data.items():
    data[key] = value.reshape(value.shape[0], -1)

    
# Print the loaded data dictionary
for key, value in data.items():
    print(key, value.shape)

index (50, 45288)
little (50, 45288)
middle (50, 45288)
ring (50, 45288)
thumb (50, 45288)


In [5]:
def train_SVM(finger1, finger2):
    print(f'Training SVM for {finger1} vs {finger2}')

    X = np.concatenate((data[finger1], data[finger2]), axis=0)
    y = np.concatenate((np.ones(data[finger1].shape[0]), np.zeros(data[finger2].shape[0])), axis=0)


    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5, 1e-6],
                        'C': [ 1, ]}]
    grid = GridSearchCV(SVC(), tuned_parameters, cv=StratifiedKFold(n_splits=10), scoring='accuracy')
    grid.fit(X, y)
    acc = grid.best_score_* 100
    print(str(round(acc, 2)))
    return acc

    

In [6]:
accuracies = []
for finger1 in data:
    for finger2 in data:
        if finger1 != finger2:
            accuracies.append(train_SVM(finger1, finger2))
        else:
            break
accucarcies = np.array(accuracies)
print(f'Average accuracy: {np.mean(accuracies)}')
print(f'Standard deviation: {np.std(accuracies)}')
            

Training SVM for little vs index
56.0
Training SVM for middle vs index
52.0
Training SVM for middle vs little
54.0
Training SVM for ring vs index
68.0
Training SVM for ring vs little
74.0
Training SVM for ring vs middle
70.0
Training SVM for thumb vs index
61.0
Training SVM for thumb vs little
52.0
Training SVM for thumb vs middle
58.0
Training SVM for thumb vs ring
78.0
Average accuracy: 62.3
Standard deviation: 9.03382532485547
